In [1]:
import numpy as np
import pandas as pd

In [2]:
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\medici\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [10]:
news_df = pd.read_csv("data/fake_news_train.csv", index_col='id')
news_df.head()

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
news_df.isna().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
news_df = news_df.fillna('')

In [13]:
news_df['article'] = news_df['title'] + news_df['author']

In [14]:
news_df

,title,author,text,label,article
id,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You FiredConsortiumnew...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...
...,...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Rapper T.I.: Trump a ’Poster Child For White S...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Macy’s Is Said to Receive Takeover Approach by...


In [15]:
news_df["author"].value_counts()

                                             1957
Pam Key                                       243
admin                                         193
Jerome Hudson                                 166
Charlie Spiering                              141
                                             ... 
Jeremy R. Hammond                               1
Vic Bishop                                      1
Douglas Martin                                  1
Najim Rahim and Fahim Abed                      1
Michael J. de la Merced and Rachel Abrams       1
Name: author, Length: 4202, dtype: int64

In [16]:
X = news_df.drop(columns='label', axis=1)
Y = news_df['label']

In [17]:
p_stemming = PorterStemmer()

In [18]:
def stemming(content):
    stemmed_word = re.sub('[^a-zA-Z]',' ',content)
    stemmed_word = stemmed_word.lower()
    stemmed_word = stemmed_word.split()
    stemmed_word = [p_stemming.stem(word) for word in stemmed_word if not word in stopwords.words('english')]
    stemmed_word = ' '.join(stemmed_word)
    return stemmed_word

In [19]:
news_df['article'] = news_df['article'].apply(stemming)

In [20]:
news_df['article']

id
0        hous dem aid even see comey letter jason chaff...
1        flynn hillari clinton big woman campu breitbar...
2                   truth might get firedconsortiumnew com
3        civilian kill singl us airstrik identifiedjess...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper trump poster child white supremaci jero...
20796    n f l playoff schedul matchup odd new york tim...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exercis balkansalex ...
20799                            keep f alivedavid swanson
Name: article, Length: 20800, dtype: object

In [21]:
X = news_df['article'].values
X

array(['hous dem aid even see comey letter jason chaffetz tweet itdarrel lucu',
       'flynn hillari clinton big woman campu breitbartdaniel j flynn',
       'truth might get firedconsortiumnew com', ...,
       'maci said receiv takeov approach hudson bay new york timesmichael j de la merc rachel abram',
       'nato russia hold parallel exercis balkansalex ansari',
       'keep f alivedavid swanson'], dtype=object)

In [22]:
Y = news_df['label'].values
Y

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [23]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [26]:
X

<20800x23585 sparse matrix of type '<class 'numpy.float64'>'
	with 196475 stored elements in Compressed Sparse Row format>

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 1)

In [28]:
ml_model = LogisticRegression()
ml_model.fit(X_train, Y_train)

LogisticRegression()

In [29]:
X_train_predict = ml_model.predict(X_train)
train_data_accuracy = accuracy_score(X_train_predict, Y_train)
percent_tr_accuracy = train_data_accuracy * 100
print("Accuracy for Train data: ", percent_tr_accuracy)

Accuracy for Train data:  98.26322115384616


In [30]:
X_test_predict = ml_model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_predict, Y_test)
percent_test_accuracy = test_data_accuracy * 100
print("Accuracy for Test data: ", percent_test_accuracy)

Accuracy for Test data:  96.5625


In [31]:
'''
index = int(input("Enter article number to be verified: "))
^ To get article number as input from user
'''

X_new = X_test[26]
new_predict = ml_model.predict(X_new)
if(new_predict[0]==0):
    print("The News is real")
else:
    print("The News is fake")

The News is fake
